# 1. Importando librerías

In [64]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2 as pg2
import csv
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, roc_curve, roc_auc_score, confusion_matrix

from helpers import *

import warnings
warnings.filterwarnings('ignore')

# 2. Leyendo archivos

In [20]:
# datos de entrenamiento
df_train = pd.read_csv('train_cupid.csv')

for col in df_train.columns:
    df_train[col] = df_train[col].astype(int)

In [21]:
# guardamos columnas
columns_all = df_train.columns

In [3]:
# datos de validacion
df_test = pd.read_csv('test_cupid.csv')

for col in df_train.columns:
    df_test[col] = df_test[col].astype(int)

In [4]:
# saving sets
df_train.to_csv('train_cupid.csv', index=False)
df_test.to_csv('test_cupid.csv', index=False)

# 3. Creando base de datos e importando tablas

In [6]:
# conectando
user = 'postgres'
password = 'moeg231@'
conn_db = pg2.connect(f"user={user} password={password}")

In [9]:
# creando BD
conn_db.autocommit = True
cursor = conn_db.cursor()
cursor.execute('CREATE DATABASE apellido_nombre')

In [26]:
# creando tablas
for table_ in ['train_data', 'test_data']:
    cursor.execute(f'create table {table_} (\
            "age" INTEGER,\
            "height" INTEGER,\
            "virgo" INTEGER,\
            "taurus" INTEGER,\
            "scorpio" INTEGER,\
            "pisces" INTEGER,\
            "libra" INTEGER,\
            "leo" INTEGER,\
            "gemini" INTEGER,\
            "aries" INTEGER,\
            "aquarius" INTEGER,\
            "cancer" INTEGER,\
            "sagittarius" INTEGER,\
            "asian" INTEGER,\
            "hispanic_latin" INTEGER,\
            "black" INTEGER,\
            "indian" INTEGER,\
            "pacific_islander" INTEGER,\
            "native_american" INTEGER,\
            "middle_eastern" INTEGER,\
            "colorado" INTEGER,\
            "new_york" INTEGER,\
            "oregon" INTEGER,\
            "arizona" INTEGER,\
            "hawaii" INTEGER,\
            "montana" INTEGER,\
            "wisconsin" INTEGER,\
            "virginia" INTEGER,\
            "spain" INTEGER,\
            "nevada" INTEGER,\
            "illinois" INTEGER,\
            "vietnam" INTEGER,\
            "ireland" INTEGER,\
            "louisiana" INTEGER,\
            "michigan" INTEGER,\
            "texas" INTEGER,\
            "united_kingdom" INTEGER,\
            "massachusetts" INTEGER,\
            "north_carolina" INTEGER,\
            "idaho" INTEGER,\
            "mississippi" INTEGER,\
            "new_jersey" INTEGER,\
            "florida" INTEGER,\
            "minnesota" INTEGER,\
            "georgia" INTEGER,\
            "utah" INTEGER,\
            "washington" INTEGER,\
            "west_virginia" INTEGER,\
            "connecticut" INTEGER,\
            "tennessee" INTEGER,\
            "rhode_island" INTEGER,\
            "district_of_columbia" INTEGER,\
            "canada" INTEGER,\
            "missouri" INTEGER,\
            "germany" INTEGER,\
            "pennsylvania" INTEGER,\
            "netherlands" INTEGER,\
            "switzerland" INTEGER,\
            "mexico" INTEGER,\
            "ohio" INTEGER,\
            "agnosticism" INTEGER,\
            "atheism" INTEGER,\
            "catholicism" INTEGER,\
            "buddhism" INTEGER,\
            "judaism" INTEGER,\
            "hinduism" INTEGER,\
            "islam" INTEGER,\
            "pro_dogs" INTEGER,\
            "pro_cats" INTEGER,\
            "spanish" INTEGER,\
            "chinese" INTEGER,\
            "french" INTEGER,\
            "german" INTEGER,\
            "single" INTEGER,\
            "seeing_someone" INTEGER,\
            "available" INTEGER,\
            "employed" INTEGER,\
            "income_between_25_50" INTEGER,\
            "income_between_50_75" INTEGER,\
            "income_over_75" INTEGER,\
            "drugs_often" INTEGER,\
            "drugs_sometimes" INTEGER,\
            "drinks_not_at_all" INTEGER,\
            "drinks_often" INTEGER,\
            "drinks_rarely" INTEGER,\
            "drinks_socially" INTEGER,\
            "drinks_very_often" INTEGER,\
            "orientation_gay" INTEGER,\
            "orientation_straight" INTEGER,\
            "sex_m" INTEGER,\
            "smokes_sometimes" INTEGER,\
            "smokes_trying_to_quit" INTEGER,\
            "smokes_when_drinking" INTEGER,\
            "smokes_yes" INTEGER,\
            "body_type_overweight" INTEGER,\
            "body_type_regular" INTEGER,\
            "education_high_school" INTEGER,\
            "education_undergrad_university" INTEGER);')

In [27]:
# set de columnas
train_set = 'train_data values ('
test_set = 'test_data values ('

for col in df_train.columns[df_train.columns != 'index']:
    if col != 'education_undergrad_university':
        train_set += '%s' + ', '
        test_set += '%s' + ', '
    else:
        train_set += '%s'
        test_set += '%s'

train_set += ')'
test_set += ')'

In [28]:
# ingestamos datos de entrenamiento y validacion a las tablas respectivas en nuestra base de datos
for set_ in ['train_cupid', 'test_cupid']:
    with open(f'./{set_}.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)
        
        if set_ == 'train_cupid':
            insert_string = train_set
        else:
            insert_string = test_set

        for row in reader:
            cursor.execute(f"INSERT INTO {insert_string}", row)

In [29]:
conn_db.commit()

# 4. Entrenamiento de los modelos

In [45]:
# conjunto de entrenamiento
cursor.execute("SELECT * FROM train_data;")
columnas_train = cursor.fetchall()
X_train = pd.DataFrame(list(columnas_train))
X_train.columns = columns_all

In [46]:
# separando variables objetivo
y_single_train = X_train['single']
y_seeing_train = X_train['seeing_someone']
y_aval_train = X_train['available']

In [47]:
# eliminando variables objetivo de la matriz de atributos
X_train = X_train.drop(['single', 'seeing_someone', 'available'], axis=1)

In [48]:
# definicion de modelos a implementar
model_dict = {
    'logistic': LogisticRegression(), 
    'dec_tree': DecisionTreeClassifier(max_depth=5), 
    'rdm_forest': RandomForestClassifier(max_depth=5),
    'grad_boost': GradientBoostingClassifier(),  
    'ada_boost': AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5)),
    'bernoulli': BernoulliNB(), 
    'svc': SVC(kernel='rbf')
    }

In [50]:
var_dict = {
    'single': y_single_train,
    'seeing': y_seeing_train,
    'aval': y_aval_train
}

In [53]:
for name_, model_ in model_dict.items():
    for var_ in ['single', 'seeing', 'aval']:
        model_temp = model_.fit(X_train, var_dict[var_])
        pickle.dump(model_temp, open(f'pickles/model_{name_}_{var_}.sav', 'wb'))

# 5. Predicciones

In [54]:
# conjunto de validacion
cursor.execute("SELECT * FROM test_data;")
columnas_test = cursor.fetchall()
X_test = pd.DataFrame(list(columnas_test))
X_test.columns = columns_all

In [55]:
# separando variables objetivo
y_single_test = X_test['single']
y_seeing_test = X_test['seeing_someone']
y_aval_test = X_test['available']

In [57]:
# eliminando variables objetivo de la matriz de atributos
X_test = X_test.drop(['single', 'seeing_someone', 'available'], axis=1)

In [56]:
var_dict_pred = {
    'single': y_single_test,
    'seeing': y_seeing_test,
    'aval': y_aval_test
}

In [58]:
mod_temp = pickle.load(open(f'pickles/model_ada_boost_aval.sav', 'rb'))

In [61]:
# realizamos predicciones y visualizamos resultados

for name_, model_instance in model_dict.items():
    for var_ in ['single', 'seeing', 'aval']:
        mod_temp = pickle.load(open(f'pickles/model_{name_}_{var_}.sav', 'rb'))
        print(f'Model: {name_} - V.O: {var_}')
        print(classification_report(var_dict_pred[var_], mod_temp.predict(X_test)))

Model: logistic - V.O: single
              precision    recall  f1-score   support

           0       0.41      0.02      0.04      1616
           1       0.92      1.00      0.96     18327

    accuracy                           0.92     19943
   macro avg       0.67      0.51      0.50     19943
weighted avg       0.88      0.92      0.88     19943

Model: logistic - V.O: seeing
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     19163
           1       0.00      0.00      0.00       780

    accuracy                           0.96     19943
   macro avg       0.48      0.50      0.49     19943
weighted avg       0.92      0.96      0.94     19943

Model: logistic - V.O: aval
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     19163
           1       0.00      0.00      0.00       780

    accuracy                           0.96     19943
   macro avg       0.48      0.50      0

# 6. Predicción de queries específicas

In [73]:
# definicion de variables a obtener en cada query
q1 = ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese']
q2 = ['income_over_75', 'french', 'german','orientation_straight', 'new_york']
q3 = ['education_undergrad_university', 'body_type_regular', 'pro_dogs', 'employed']
q4 = ['taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism']

In [93]:
table_creation_dict = {
    'query_1': 'create table {} ("atheism" INTEGER, "asian" INTEGER, "employed" INTEGER, "pro_dogs" INTEGER, "chinese" INTEGER, "single_yhat" REAL);',
    'query_2': 'create table {} ("income_over_75" INTEGER, "french" INTEGER, "german" INTEGER, "orientation_straight" INTEGER, "new_york" INTEGER, "single_yhat" REAL);',
    'query_3': 'create table {} ("education_undergrad_university" INTEGER, "body_type_regular" INTEGER, "pro_dogs" INTEGER, "employed" INTEGER, "single_yhat" REAL);',
    'query_4': 'create table {} ("taurus" INTEGER, "indian" INTEGER, "washington" INTEGER, "income_between_50_75" INTEGER, "hinduism" INTEGER, "single_yhat" REAL);'
}


In [102]:
def insert_values(table_name, structure_, df_temp):
    for index, row in df_temp.iterrows():
        cursor.execute(f"INSERT INTO {table_name} VALUES {structure_}", row.values)

In [105]:
# calculamos los perfiles de acuerdo a cada grupo de variables e ingestamos en la BD

for name_, model_instance in model_dict.items():
    for var_ in ['single', 'seeing', 'aval']:
        for query_ in [q1, q2, q3, q4]:
            print(f'Model: {name_} - V.O: {var_} - Query: {query_}')
            # creacion de perfiles
            df_temp = pd.DataFrame(create_crosstab(f'pickles/model_{name_}_{var_}.sav', X_test, var_dict_pred[var_], query_)).reset_index()
            # creacion de tablas en BD e ingestamos datos
            if query_ == q1:
                cursor.execute(table_creation_dict['query_1'].format(f'{name_}_{var_}_query_1'))
                insert_values(f'{name_}_{var_}_query_1', '(%s, %s, %s, %s, %s, %s)', df_temp)
            elif query_ == q2:
                cursor.execute(table_creation_dict['query_1'].format(f'{name_}_{var_}_query_2'))
                insert_values(f'{name_}_{var_}_query_2', '(%s, %s, %s, %s, %s, %s)', df_temp)
            elif query_ == q3:
                cursor.execute(table_creation_dict['query_1'].format(f'{name_}_{var_}_query_3'))
                insert_values(f'{name_}_{var_}_query_3', '(%s, %s, %s, %s, %s)', df_temp)
            else:
                cursor.execute(table_creation_dict['query_1'].format(f'{name_}_{var_}_query_4'))
                insert_values(f'{name_}_{var_}_query_4', '(%s, %s, %s, %s, %s, %s)', df_temp)

Model: logistic - V.O: single - Query: ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese']
Model: logistic - V.O: single - Query: ['income_over_75', 'french', 'german', 'orientation_straight', 'new_york']
Model: logistic - V.O: single - Query: ['education_undergrad_university', 'body_type_regular', 'pro_dogs', 'employed']
Model: logistic - V.O: single - Query: ['taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism']
Model: logistic - V.O: seeing - Query: ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese']
Model: logistic - V.O: seeing - Query: ['income_over_75', 'french', 'german', 'orientation_straight', 'new_york']
Model: logistic - V.O: seeing - Query: ['education_undergrad_university', 'body_type_regular', 'pro_dogs', 'employed']
Model: logistic - V.O: seeing - Query: ['taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism']
Model: logistic - V.O: aval - Query: ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese']
Model: logistic - V.O: aval - Q

In [106]:
conn_db.commit()
cursor.close()

# 7. Exportación base de datos